An HR analyst in Unitech Pvt Ltd, wants to predict the annual salaries of given employees using the potential explanatory variables.

1) Estimate the appropriate multiple linear regression equation to predict the salary of an Unitech employee using all explanatory variables<br>
2) Do we need to exclude certain columns? Why?<br>
3) Which department employees are paid the highest? By how much?<br>
4) Do you see any discrimination in salaries earned by male and female employees?<br>
5) What would be the estimated Salary of a Sr. Data Scientist (joining engineering) with 10 years of work experience. This woman has 18 years of total education, and will be supervising 4 junior employees.

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

In [2]:
# load data
df = pd.read_csv('C:/Users/Karthik.Iyer/Downloads/AccelerateAI/Regression-Models-main/MLR_Q13_EmpSalary.csv')
df.head()

,Employee,Salary,PreviousExp,YearsEmployed,YearsEducation,DirectRepotees,Female,Male,Engineering,Sales,Other
0,1,"$65,487",0,27,22,44,0,1,1,0,0
1,2,"$46,184",3,20,14,1,1,0,1,0,0
2,3,"$32,782",1,0,17,0,1,0,0,1,0
3,4,"$54,899",5,12,18,0,0,1,1,0,0
4,5,"$34,869",5,7,14,1,0,1,1,0,0


In [3]:
# Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Employee        46 non-null     int64 
 1   Salary          46 non-null     object
 2   PreviousExp     46 non-null     int64 
 3   YearsEmployed   46 non-null     int64 
 4   YearsEducation  46 non-null     int64 
 5   DirectRepotees  46 non-null     int64 
 6   Female          46 non-null     int64 
 7   Male            46 non-null     int64 
 8   Engineering     46 non-null     int64 
 9   Sales           46 non-null     int64 
 10  Other           46 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 4.1+ KB


We need to handle for Salary by removing the $ and , characters. All other data types look appropriate.

In [4]:
# Check for missing values
df.isnull().sum()

Employee          0
Salary            0
PreviousExp       0
YearsEmployed     0
YearsEducation    0
DirectRepotees    0
Female            0
Male              0
Engineering       0
Sales             0
Other             0
dtype: int64

In [5]:
# Check the shape
df.shape

(46, 11)

There are no missing values.

In [6]:
# Salary
df['Salary'] = df['Salary'].apply(lambda x: int(x.replace('$','').replace(',','')))

In [7]:
# Check data
df.head()

,Employee,Salary,PreviousExp,YearsEmployed,YearsEducation,DirectRepotees,Female,Male,Engineering,Sales,Other
0,1,65487,0,27,22,44,0,1,1,0,0
1,2,46184,3,20,14,1,1,0,1,0,0
2,3,32782,1,0,17,0,1,0,0,1,0
3,4,54899,5,12,18,0,0,1,1,0,0
4,5,34869,5,7,14,1,0,1,1,0,0


In [8]:
# Drop Employee as it is just unique number
df.drop('Employee', axis=1, inplace=True)

In [9]:
# Check columns
df.columns

Index(['Salary', 'PreviousExp', 'YearsEmployed', 'YearsEducation',
       'DirectRepotees', 'Female', 'Male', 'Engineering', 'Sales', 'Other'],
      dtype='object')

In [10]:
# Female, Male categories can be merged into one column called Gender. Either it can be Female or Male

df.loc[df['Female'] == 1, 'Gender'] = 'Female'
df.loc[df['Female'] == 0, 'Gender'] = 'Male'

In [11]:
# Check data
df.head()

,Salary,PreviousExp,YearsEmployed,YearsEducation,DirectRepotees,Female,Male,Engineering,Sales,Other,Gender
0,65487,0,27,22,44,0,1,1,0,0,Male
1,46184,3,20,14,1,1,0,1,0,0,Female
2,32782,1,0,17,0,1,0,0,1,0,Female
3,54899,5,12,18,0,0,1,1,0,0,Male
4,34869,5,7,14,1,0,1,1,0,0,Male


In [12]:
# Engineering, Sales, Other categories can be merged into one column called Department

df.loc[df['Engineering'] == 1, 'Department'] = 'Engineering'
df.loc[df['Sales'] == 1, 'Department'] = 'Sales'
df.loc[df['Other'] == 1, 'Department'] = 'Other'

In [13]:
# Check data
df.sample(5)

,Salary,PreviousExp,YearsEmployed,YearsEducation,DirectRepotees,Female,Male,Engineering,Sales,Other,Gender,Department
30,69246,3,22,20,45,1,0,1,0,0,Female,Engineering
10,23654,0,0,10,2,0,1,0,0,1,Male,Other
34,35631,4,6,14,2,1,0,1,0,0,Female,Engineering
43,31425,6,7,15,6,0,1,0,0,1,Male,Other
27,51698,6,18,16,1,1,0,1,0,0,Female,Engineering


In [14]:
# Drop the redundant variables

df.drop(columns=['Female', 'Male', 'Engineering', 'Sales', 'Other'], axis=1, inplace=True)
df.columns

Index(['Salary', 'PreviousExp', 'YearsEmployed', 'YearsEducation',
       'DirectRepotees', 'Gender', 'Department'],
      dtype='object')

In [15]:
# Create dummy variables for Gender and Department
df_dummy_Gender = pd.get_dummies(df.Gender, drop_first=True)
df_dummy_Dptmt = pd.get_dummies(df.Department, drop_first=True)

In [16]:
# Merge with main data frame
df = pd.concat([df, df_dummy_Gender, df_dummy_Dptmt], axis=1)
df.head()

,Salary,PreviousExp,YearsEmployed,YearsEducation,DirectRepotees,Gender,Department,Male,Other,Sales
0,65487,0,27,22,44,Male,Engineering,1,0,0
1,46184,3,20,14,1,Female,Engineering,0,0,0
2,32782,1,0,17,0,Female,Sales,0,0,1
3,54899,5,12,18,0,Male,Engineering,1,0,0
4,34869,5,7,14,1,Male,Engineering,1,0,0


In [17]:
# Drop the original variables from which dummies were created
df.drop(columns=['Gender', 'Department'], axis=1, inplace=True)
df.columns

Index(['Salary', 'PreviousExp', 'YearsEmployed', 'YearsEducation',
       'DirectRepotees', 'Male', 'Other', 'Sales'],
      dtype='object')

In [18]:
# Check data
df.head()

,Salary,PreviousExp,YearsEmployed,YearsEducation,DirectRepotees,Male,Other,Sales
0,65487,0,27,22,44,1,0,0
1,46184,3,20,14,1,0,0,0
2,32782,1,0,17,0,0,0,1
3,54899,5,12,18,0,1,0,0
4,34869,5,7,14,1,1,0,0


### 1) Estimate the appropriate multiple linear regression equation to predict the salary of an Unitech employee using all explanatory variables

In [19]:
# Lets train the model
Y = df['Salary']
X = df.drop('Salary', axis=1)

X = sm.add_constant(X) #adding constant
reg_model = sm.OLS(Y,X).fit()
reg_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     29.78
Date:                Wed, 18 May 2022   Prob (F-statistic):           1.48e-13
Time:                        22:48:07   Log-Likelihood:                -449.82
No. Observations:                  46   AIC:                             915.6
Df Residuals:                      38   BIC:                             930.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.149e+04   4823.228      2.382      0.022    1725.378    2.13e+04
PreviousExp      -61.4031    200.339     -0.306      0.761    -466.967     344.161
YearsEmployed    664.6281    117.548      5.654      0.000     426.664     902.592
YearsEducation  1599.1368    339.386      4.712      0.000     912.086    2286.187
DirectRepotees   130.0708     82.554      1.576      0.123     -37.050     297.192
Male           -1882.4467   1459.718     -1.290      0.205   -4837.490    1072.597
Other          -1277.3730   1612.197     -0.792      0.433   -4541.094    1986.348
Sales          -8084.3403   1850.239     -4.369      0.000   -1.18e+04   -4338.728
==============================================================================
Omnibus:                        0.067   Durbin-Watson:                   2.720
Prob(Omnibus):                  0.967   Jarque-Bera (JB):                0.225
Skew:                           0.073   Prob(JB):                        0.894
Kurtosis:                       2.690   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# Consider only significant variables
reg_model.pvalues[reg_model.pvalues < 0.05]

const             0.022323
YearsEmployed     0.000002
YearsEducation    0.000033
Sales             0.000093
dtype: float64

In [21]:
# Lets check multi-collinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)

const             48.448358
PreviousExp        1.135598
YearsEmployed      1.626557
YearsEducation     1.891932
DirectRepotees     1.450076
Male               1.107285
Other              1.189534
Sales              1.212958
dtype: float64

Leaving the constant, the VIF values are below 5 indicating very less multi-collinearity

In [22]:
# Train the model with only significant variables

Y = df['Salary']
X = df[['YearsEmployed', 'YearsEducation', 'Sales']]

X = sm.add_constant(X)
reg_model = sm.OLS(Y,X).fit()
reg_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                     66.11
Date:                Wed, 18 May 2022   Prob (F-statistic):           5.93e-16
Time:                        22:48:07   Log-Likelihood:                -452.70
No. Observations:                  46   AIC:                             913.4
Df Residuals:                      42   BIC:                             920.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           6285.0605   4205.984      1.494      0.143   -2202.958    1.48e+04
YearsEmployed    699.5965    117.664      5.946      0.000     462.141     937.052
YearsEducation  1858.2705    314.845      5.902      0.000    1222.888    2493.653
Sales          -7714.2609   1703.733     -4.528      0.000   -1.12e+04   -4275.989
==============================================================================
Omnibus:                        0.156   Durbin-Watson:                   2.704
Prob(Omnibus):                  0.925   Jarque-Bera (JB):                0.002
Skew:                          -0.006   Prob(JB):                        0.999
Kurtosis:                       2.975   Cond. No.                         116.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Regression Equation:**<br>
Salary = 6285.0605 + 699.5965 * YearsEmployed + 1858.2705 * YearsEducation - 7714.2609 * Sales

### 2) Do we need to exclude certain columns? Why?

Yes. Only statistically significant variables whose p-value is less than 0.05 have been included in the model, and the rest have been excluded.

### 3) Which department employees are paid the highest? By how much?

We have only Sales department which is significant in the model. The coefficient beta (-7714.2609) indicate that Salary of employees in Sales department are paid lesser than those in the Engineering department (base category) by 7714.2609.

But we cannot make any conclusion here as we have excluded the other departments which were statistically insignificant.

### 4) Do you see any discrimination in salaries earned by male and female employees?

We cannot determine this as these variables are statistically insignificant and have been excluded in the model.

### 5) What would be the estimated Salary of a Sr. Data Scientist (joining engineering) with 10 years of work experience. This woman has 18 years of total education, and will be supervising 4 junior employees.

This cannot be determined as the engineering department is no more significant in the model due to p-value less than 0.05 and has been excluded.